In [1]:
# import libraries
import os
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

In [2]:
# directories
main_dir = os.path.join(os.getcwd())
data_dir = os.path.join(main_dir, "Data")

In [3]:
# load data 
precovid_data = pd.read_csv(os.path.join(data_dir, 'precovid_crime_micro_geo_11_29_2021.csv'))
covid_data = pd.read_csv(os.path.join(data_dir, 'covid_crime_micro_geo_11_29_2021.csv'))

In [4]:
precovid_data = precovid_data.drop(columns=['Unnamed: 0'])
covid_data = covid_data.drop(columns=['Unnamed: 0'])

In [5]:
precovid_data['date'] = pd.to_datetime(precovid_data['date'])
covid_data['date'] = pd.to_datetime(covid_data['date'])

In [6]:
# restrict pre and during covid timeframes
precovid_restriction = pd.to_datetime("2019-11-30").date()
covid_restriction = pd.to_datetime("2020-07-01").date()

precovid_data = precovid_data.loc[precovid_data['date'].dt.date > precovid_restriction]
covid_data = covid_data.loc[covid_data['date'].dt.date <= covid_restriction]

In [7]:
# # shift one hot encode
# shift_dummies_precovid = pd.get_dummies(precovid_data.SHIFT)
# precovid_data = pd.concat([precovid_data, shift_dummies_precovid], axis=1)

# shift_dummies_covid = pd.get_dummies(covid_data.SHIFT)
# covid_data = pd.concat([covid_data, shift_dummies_covid], axis=1)

# # method one hot encode
# shift_dummies_precovid = pd.get_dummies(precovid_data.METHOD)
# precovid_data = pd.concat([precovid_data, shift_dummies_precovid], axis=1)

# shift_dummies_covid = pd.get_dummies(covid_data.METHOD)
# covid_data = pd.concat([covid_data, shift_dummies_covid], axis=1)

In [8]:
precovid_data.columns

Index(['SHIFT', 'OFFENSE', 'WARD', 'LATITUDE', 'LONGITUDE', 'date',
       'total_cases', 'daily_cases', 'daily_death',
       'OUT_POSITIVE_QUARANTINE_PSMPD', 'MO', 'TU', 'WE', 'TH', 'FR', 'SA',
       'SU', 'JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEP',
       'OCT', 'NOV', 'DEC', 'ward_1', 'ward_2', 'ward_3', 'ward_4', 'ward_5',
       'ward_6', 'ward_7', 'ward_8', 'ARSON', 'ASSAULT W/DANGEROUS WEAPON',
       'BURGLARY', 'HOMICIDE', 'MOTOR VEHICLE THEFT', 'ROBBERY', 'SEX ABUSE',
       'THEFT F/AUTO', 'THEFT/OTHER', 'DAY', 'EVENING', 'MIDNIGHT',
       'distance_metro', 'distance_nightclubs', 'distance_worship'],
      dtype='object')

In [9]:
# drop shift, offense, date columns
precovid_data_baseline = precovid_data.drop(columns=['daily_cases', 'daily_death', 'OUT_POSITIVE_QUARANTINE_PSMPD',
                                                     'total_cases', 'SHIFT', 'OFFENSE', 'date', 'LATITUDE', 'LONGITUDE', 
                                                     'ward_1', 'ward_2', 'ward_3', 'ward_4', 'ward_5', 
                                                     'ward_6', 'ward_7', 'ward_8'])
covid_data_baseline = covid_data.drop(columns=['SHIFT', 'OFFENSE', 'date', 'LATITUDE', 'LONGITUDE', 'total_cases', 
                                            'ward_1', 'ward_2', 'ward_3', 'ward_4', 'ward_5', 'ward_6', 
                                            'ward_7', 'ward_8'])

In [10]:
# declare target and features
y_precovid_base = precovid_data_baseline.pop('WARD') 
X_precovid_base = precovid_data_baseline

y_covid_base = covid_data_baseline.pop('WARD') 
X_covid_base = covid_data_baseline

In [11]:
# train test split
X_precovid_train, X_precovid_test, y_precovid_train, y_precovid_test = train_test_split(X_precovid_base, 
                                                                                        y_precovid_base, 
                                                                                        test_size=0.3, 
                                                                                        random_state=42)

X_covid_train, X_covid_test, y_covid_train, y_covid_test = train_test_split(X_covid_base, 
                                                                            y_covid_base, 
                                                                            test_size=0.3, 
                                                                            random_state=42)

In [12]:
# scale data - MinMax
scaler = MinMaxScaler(feature_range=(0, 1))
scaler = scaler.fit(X_precovid_train)
X_precovid_train_scaled = scaler.transform(X_precovid_train)
X_precovid_test_scaled = scaler.transform(X_precovid_test)

scaler = MinMaxScaler(feature_range=(0, 1))
scaler = scaler.fit(X_covid_train)
X_covid_train_scaled = scaler.transform(X_covid_train)
X_covid_test_scaled = scaler.transform(X_covid_test)

# scale data - Standard Scaler
scaler2 = StandardScaler()
scaler2 = scaler2.fit(X_precovid_train)
X_precovid_train_scaled2 = scaler2.transform(X_precovid_train)
X_precovid_test_scaled2 = scaler2.transform(X_precovid_test)

scaler2 = StandardScaler()
scaler2 = scaler2.fit(X_covid_train)
X_covid_train_scaled2 = scaler2.transform(X_covid_train)
X_covid_test_scaled2 = scaler2.transform(X_covid_test)

## Run models with Min Max Scaler

In [13]:
# instantiate model precovid
lr_precovid = LogisticRegression(multi_class='multinomial', max_iter=500)

# precovid train
lr_precovid.fit(X_precovid_train_scaled, y_precovid_train)
# make predictions
y_precovid_pred = lr_precovid.predict(X_precovid_test_scaled)
# score
precovid_score = lr_precovid.score(X_precovid_test_scaled, y_precovid_test)
print(precovid_score)

0.43439911797133407


In [14]:
# instantiate model covid
lr_covid = LogisticRegression(multi_class='multinomial', max_iter=500)

# covid train
lr_covid.fit(X_covid_train_scaled, y_covid_train)
# make predictions
y_covid_pred = lr_covid.predict(X_covid_test_scaled)
# score
covid_score = lr_covid.score(X_covid_test_scaled, y_covid_test)
print(covid_score)

0.4266098484848485


## Run Models with Standard Scaler

In [15]:
# instantiate model precovid
lr_precovid = LogisticRegression(multi_class='multinomial', max_iter=500)

# precovid train
lr_precovid.fit(X_precovid_train_scaled2, y_precovid_train)
# make predictions
y_precovid_pred = lr_precovid.predict(X_precovid_test_scaled2)
# score
precovid_score = lr_precovid.score(X_precovid_test_scaled2, y_precovid_test)
print(precovid_score)

0.44358691657478866


In [16]:
# instantiate model covid
lr_covid = LogisticRegression(multi_class='multinomial', max_iter=500)

# covid train
lr_covid.fit(X_covid_train_scaled2, y_covid_train)
# make predictions
y_covid_pred = lr_covid.predict(X_covid_test_scaled2)
# score
covid_score = lr_covid.score(X_covid_test_scaled2, y_covid_test)
print(covid_score)

0.4337121212121212


In [17]:
grid = {"C":np.logspace(-3,3,7), 
        "penalty":["l1","l2","elasticnet"], 
        "solver":['newton-cg','lbfgs','sag']}

# instantiate model
lr_precovid = LogisticRegression(multi_class='multinomial')
# precovid
lr_precovid_CV = GridSearchCV(lr_precovid, grid, cv=3)
# fit precovid
lr_precovid_CV.fit(X_precovid_train_scaled, y_precovid_train)
# make predictions
y_precovid_pred = lr_precovid_CV.predict(X_precovid_test_scaled)


/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 1304, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 442, in _check_solver
    raise ValueError("Solver %s supports only 'l2' or 'none' penalties, "
ValueError: Solver newton-cg supports only 'l2' or 'none' penalties, got l1 penalty.

  warnings.warn("Estimator fit failed. The score on this train-test"
/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selectio

/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 1304, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 442, in _check_solver
    raise ValueError("Solver %s supports only 'l2' or 'none' penalties, "
ValueError: Solver newton-cg supports only 'l2' or 'none' penalties, got elasticnet penalty.

  warnings.warn("Estimator fit failed. The score on this train-test"
/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_

/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 1304, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 442, in _check_solver
    raise ValueError("Solver %s supports only 'l2' or 'none' penalties, "
ValueError: Solver newton-cg supports only 'l2' or 'none' penalties, got elasticnet penalty.

  warnings.warn("Estimator fit failed. The score on this train-test"
/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_

/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

{'C': 1000.0, 'penalty': 'l2', 'solver': 'newton-cg'} 0.4331166645066818


In [27]:
print(lr_precovid_CV.best_params_, lr_precovid_CV.best_score_)

{'C': 1000.0, 'penalty': 'l2', 'solver': 'newton-cg'} 0.4331166645066818


In [18]:
# instantiate model
lr_covid = LogisticRegression(multi_class='multinomial')
# precovid
lr_covid_CV = GridSearchCV(lr_covid, grid, cv=3)
# fit precovid
lr_covid_CV.fit(X_covid_train_scaled, y_covid_train)
# make predictions
y_covid_pred = lr_covid_CV.predict(X_covid_test_scaled)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 1304, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 442, in _check_solver
    raise ValueError("Solver %s supports only 'l2' or 'none' penalties, "
ValueError: Solver newton-cg supports only 'l2' or 'none' penalties, got l1 penalty.

  warnings.warn("Estimator fit failed. The score on this train-test"
/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selectio

/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 1304, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 442, in _check_solver
    raise ValueError("Solver %s supports only 'l2' or 'none' penalties, "
ValueError: Solver newton-cg supports only 'l2' or 'none' penalties, got elasticnet penalty.

  warnings.warn("Estimator fit failed. The score on this train-test"
/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_

/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 1304, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 442, in _check_solver
    raise ValueError("Solver %s supports only 'l2' or 'none' penalties, "
ValueError: Solver newton-cg supports only 'l2' or 'none' penalties, got elasticnet penalty.

  warnings.warn("Estimator fit failed. The score on this train-test"
/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_

/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

{'C': 1000.0, 'penalty': 'l2', 'solver': 'lbfgs'} 0.44478278522127485


/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 1304, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 442, in _check_solver
    raise ValueError("Solver %s supports only 'l2' or 'none' penalties, "
Val

In [28]:
print(lr_covid_CV.best_params_, lr_covid_CV.best_score_)

{'C': 1000.0, 'penalty': 'l2', 'solver': 'lbfgs'} 0.44478278522127485


In [19]:
# # best precovid model
# lr_precovid_best = LogisticRegression(multi_class='multinomial', C=10, penalty='l2', solver='newton-cg')
# # precovid train
# lr_precovid_best.fit(X_precovid_train_scaled, y_precovid_train)


# # make predictions
# # y_precovid_pred_best = lr_precovid_best.predict(X_precovid_test_scaled)
# # score
# # precovid_score = lr_precovid_best.score(X_precovid_test_scaled, y_precovid_test)
# # print(precovid_score)

In [20]:
# # best covid model
# lr_covid_best = LogisticRegression(multi_class='multinomial', C=1000, penalty='l2', solver='lbfgs')
# # covid train
# lr_covid_best.fit(X_covid_train_scaled, y_covid_train)


# # # make predictions
# # y_covid_pred_best = lr_covid_best.predict(X_covid_test_scaled)
# # # score
# # covid_score = lr_covid_best.score(X_covid_test_scaled, y_covid_test)
# # print(covid_score)

# Experiment

In [21]:
# experiment datasets
# precovid exp data now HAS the covid columns -- all values are 0
precovid_data_exp = precovid_data.drop(columns=['total_cases', 'SHIFT', 'OFFENSE', 'date', 'LATITUDE', 'LONGITUDE', 
                                                     'ward_1', 'ward_2', 'ward_3', 'ward_4', 'ward_5', 
                                                     'ward_6', 'ward_7', 'ward_8'])
# covid exp data doest NOT have covid columns
covid_data_exp = covid_data.drop(columns=['daily_cases', 'daily_death', 'OUT_POSITIVE_QUARANTINE_PSMPD',
                                                     'total_cases', 'SHIFT', 'OFFENSE', 'date', 'LATITUDE', 'LONGITUDE', 
                                                     'ward_1', 'ward_2', 'ward_3', 'ward_4', 'ward_5', 
                                                     'ward_6', 'ward_7', 'ward_8'])

In [22]:
# declare target and features for EXP
y_precovid_exp = precovid_data_exp.pop('WARD') 
X_precovid_exp = precovid_data_exp

y_covid_exp = covid_data_exp.pop('WARD') 
X_covid_exp = covid_data_exp

In [23]:
# train test split for EXP
X_precovid_train_EXP, X_precovid_test_EXP, y_precovid_train_EXP, y_precovid_test_EXP = train_test_split(
                                                                                        X_precovid_exp, 
                                                                                        y_precovid_exp, 
                                                                                        test_size=0.3, 
                                                                                        random_state=42)

X_covid_train_EXP, X_covid_test_EXP, y_covid_train_EXP, y_covid_test_EXP = train_test_split(X_covid_exp, 
                                                                            y_covid_exp, 
                                                                            test_size=0.3, 
                                                                            random_state=42)

In [24]:
# SCALE EXP DATA
# scale data - MinMax
scaler = MinMaxScaler(feature_range=(0, 1))
scaler = scaler.fit(X_precovid_train_EXP)
X_precovid_train_scaled_EXP = scaler.transform(X_precovid_train_EXP)
X_precovid_test_scaled_EXP = scaler.transform(X_precovid_test_EXP)

scaler = MinMaxScaler(feature_range=(0, 1))
scaler = scaler.fit(X_covid_train_EXP)
X_covid_train_scaled_EXP = scaler.transform(X_covid_train_EXP)
X_covid_test_scaled_EXP = scaler.transform(X_covid_test_EXP)

# scale data - Standard Scaler
scaler2 = StandardScaler()
scaler2 = scaler2.fit(X_precovid_train_EXP)
X_precovid_train_scaled2_EXP = scaler2.transform(X_precovid_train_EXP)
X_precovid_test_scaled2_EXP = scaler2.transform(X_precovid_test_EXP)

scaler2 = StandardScaler()
scaler2 = scaler2.fit(X_covid_train_EXP)
X_covid_train_scaled2_EXP = scaler2.transform(X_covid_train_EXP)
X_covid_test_scaled2_EXP = scaler2.transform(X_covid_test_EXP)

In [25]:
# precovid experiment 
# predict
y_precovid_pred_EXP = lr_precovid_CV.predict(X_covid_test_scaled_EXP)
# score
precovid_score_EXP = lr_precovid_CV.score(X_covid_test_scaled_EXP, y_covid_test_EXP)
print(precovid_score_EXP)

0.4228219696969697


In [26]:
# covid experiment
# predict
y_covid_pred_EXP = lr_covid_CV.predict(X_precovid_test_scaled_EXP)
# score
covid_score_EXP = lr_covid_CV.score(X_precovid_test_scaled_EXP, y_precovid_test_EXP)
print(covid_score_EXP)

0.4355016538037486
